In [ ]:
%matplotlib inline

import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC

from utils import (
    generate_hybrid_dataset,
    make_kernel_circuit,
    make_noisy_kernel_circuit,
    compute_kernel_matrix,
    kernel_spectral_entropy,
    kernel_matrix_rank,
    plot_metric_curve,
    numerical_gradient,
)

RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)

In [ ]:
X, y, meta = generate_hybrid_dataset(n_samples=500, seed=RANDOM_SEED)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=RANDOM_SEED, stratify=y
)

X_train.shape, X_test.shape

In [ ]:
omega_values = np.linspace(1.0, 20.0, 10)
accs_ideal, ranks_ideal, ents_ideal = [], [], []
accs_noisy, ents_noisy = [], []

In [ ]:
for omega in omega_values:
    print(f"ω = {omega:.2f}")

    kernel_fn = make_kernel_circuit(n_qubits=4, omega=omega)
    K_train = compute_kernel_matrix(kernel_fn, X_train, X_train)
    K_test = compute_kernel_matrix(kernel_fn, X_test, X_train)

    clf = SVC(kernel="precomputed")
    clf.fit(K_train, y_train)
    y_pred = clf.predict(K_test)
    acc = accuracy_score(y_test, y_pred)
    accs_ideal.append(acc)

    rnk = kernel_matrix_rank(K_train)
    ranks_ideal.append(rnk)

    ent = kernel_spectral_entropy(K_train)
    ents_ideal.append(ent)

    noisy_kernel_fn = make_noisy_kernel_circuit(n_qubits=4, omega=omega, noise_level=0.01)
    K_train_n = compute_kernel_matrix(noisy_kernel_fn, X_train, X_train)
    K_test_n = compute_kernel_matrix(noisy_kernel_fn, X_test, X_train)

    clf_n = SVC(kernel="precomputed")
    clf_n.fit(K_train_n, y_train)
    y_pred_n = clf_n.predict(K_test_n)
    acc_n = accuracy_score(y_test, y_pred_n)
    accs_noisy.append(acc_n)

    ent_n = kernel_spectral_entropy(K_train_n)
    ents_noisy.append(ent_n)

In [ ]:
plot_metric_curve(
    accs_ideal,
    omega_values,
    xlabel="ω (encoding frequency)",
    ylabel="Accuracy",
    title="Quantum Kernel Accuracy vs ω (Ideal)",
)

plot_metric_curve(
    accs_noisy,
    omega_values,
    xlabel="ω (encoding frequency)",
    ylabel="Accuracy",
    title="Quantum Kernel Accuracy vs ω (Noisy, p=0.01)",
)

In [ ]:
plot_metric_curve(
    ranks_ideal,
    omega_values,
    xlabel="ω (encoding frequency)",
    ylabel="Rank(K_train)",
    title="Kernel Matrix Rank vs ω (Ideal)",
)

In [ ]:
plot_metric_curve(
    ents_ideal,
    omega_values,
    xlabel="ω (encoding frequency)",
    ylabel="Spectral Entropy",
    title="Kernel Spectral Entropy vs ω (Ideal)",
)

plot_metric_curve(
    ents_noisy,
    omega_values,
    xlabel="ω (encoding frequency)",
    ylabel="Spectral Entropy",
    title="Kernel Spectral Entropy vs ω (Noisy, p=0.01)",
)

In [ ]:
rank_grad = numerical_gradient(ranks_ideal, omega_values)
entropy_grad = numerical_gradient(ents_ideal, omega_values)

plot_metric_curve(rank_grad, omega_values, "ω", "dRank/dω", "Rank Derivative")
plot_metric_curve(entropy_grad, omega_values, "ω", "dEntropy/dω", "Spectral Entropy Derivative")